### Skew minima

In [8]:
def MinimumSkew(Genome):
    res=list()
    cntr=0
    min=0
    min_in=[]
    res.append(cntr)
    for i in range(0, len(Genome)-1):
        if Genome[i] == 'C':
            cntr-=1
        elif Genome[i] == 'G':
            cntr+=1
        res.append(cntr)
        if cntr<min:
            min=cntr
            min_in=[i+1]
        elif cntr==min:
            min_in.append(i+1)
    return min_in

In [3]:
MinimumSkew('TAAAGACTGCCGAGAGGCCAACACGAGTGCTAGAACGAGGGGCGTAAACGCGGGTCCGAT')

[11, 24]

### Hamming distance

In [9]:
def HammingDistance(p, q):
    cntr = 0
    for i in range(0, len(p)):
        if p[i] != q[i]:
            cntr+=1
    return cntr

In [5]:
HammingDistance('GGGCCGTTGGT', 'GGACCGTTGAC')

3

<code>ApproximatePatternCount(Text, Pattern, d)
        count ← 0
        for i ← 0 to |Text| − |Pattern|
            Pattern′ ← Text(i , |Pattern|)
            if HammingDistance(Pattern, Pattern′) ≤ d
                count ← count + 1
        return count</code>

In [10]:
def ApproximatePatternCount(Text, Pattern, d):
    count=0
    l = len(Pattern)
    for i in range(0, len(Text)-l+1):
        if HammingDistance(Text[i: i+l], Pattern)<=d:
            count+=1
    return count

In [7]:
ApproximatePatternCount('CGCCCGAATCCAGAACGCATTCCCATATTTCGGGACCACTGGCCTCCACGGTACGGACGTCAATCAAAT', 'ATTCTGGA', 3)

4

<code>ImmediateNeighbors(Pattern)
        Neighborhood ← the set consisting of single string Pattern
        for i = 1 to |Pattern|
            symbol ← i-th nucleotide of Pattern
            for each nucleotide x different from symbol
                Neighbor ← Pattern with the i-th nucleotide substituted by x
                add Neighbor to Neighborhood
        return Neighborhood</code>

In [11]:
def ImmediateNeighbors(Pattern):
    neighborhood = set()
    neighborhood.add(Pattern)
    for i in range(0, len(Pattern)):
        symbol = Pattern[i]
        not_sym = [x for x in ['A', 'T', 'G', 'C'] if x!=symbol]
        for x in not_sym:
            neighbor = Pattern[:i]+x+Pattern[i+1:]
            neighborhood.add(neighbor)
    return neighborhood

In [9]:
ImmediateNeighbors('ACG')

{'AAG', 'ACA', 'ACC', 'ACG', 'ACT', 'AGG', 'ATG', 'CCG', 'GCG', 'TCG'}

<code>Neighbors(Pattern, d)
        if d = 0
            return {Pattern}
        if |Pattern| = 1 
            return {A, C, G, T}
        Neighborhood ← an empty set
        SuffixNeighbors ← Neighbors(Suffix(Pattern), d)
        for each string Text from SuffixNeighbors
            if HammingDistance(Suffix(Pattern), Text) < d
                for each nucleotide x
                    add x • Text to Neighborhood
            else
                add FirstSymbol(Pattern) • Text to Neighborhood
        return Neighborhood</code>

In [12]:
def Neighbors(Pattern, d):
    neighborhood = set()
    if d==0:
        return neighborhood.add(Pattern)
    if len(Pattern) == 1:
        x=set()
        x.add('A')
        x.add('C')
        x.add('G')
        x.add('T')
        return x
    suffix_neighbors = Neighbors(Suffix(Pattern), d)
    for text in suffix_neighbors:
        if HammingDistance(Suffix(Pattern), text) < d:
            for x in ['A', 'C', 'G', 'T']:
                neighborhood.add(x+text)
        else:
            neighborhood.add(Pattern[0]+text)
    return neighborhood
    
def Suffix(Pattern):
    return Pattern[1:]

In [21]:
Neighbors('ACG', 1)

{'AAG', 'ACA', 'ACC', 'ACG', 'ACT', 'AGG', 'ATG', 'CCG', 'GCG', 'TCG'}

<code>IterativeNeighbors(Pattern, d)
        Neighborhood ← set consisting of single string Pattern
        for j = 1 to d
            for each string Pattern’ in Neighborhood
                add ImmediateNeighbors(Pattern') to Neighborhood
                remove duplicates from Neighborhood
        return Neighborhood</code>

In [13]:
def IterativeNeighbors(Pattern, d):
    neighborhood = set()
    neighborhood.add(Pattern)
    for j in range(1, d):
        for pattern in neighborhood:
            pattern.add(ImmediateNeighbors(pattern))
    return neighborhood

<code>ComputingFrequenciesWithMismatches(Text, k, d)
    for i ← 0 to 4^k − 1
        FrequencyArray(i) ← 0
    for i ← 0 to |Text|−k
        Pattern ← Text(i, k)
        Neighborhood ← Neighbors(Pattern, d)
        for each string ApproximatePattern in Neighborhood
            j ← PatternToNumber(ApproximatePattern)
            FrequencyArray(j) ← FrequencyArray(j) + 1
    return FrequencyArray </code>

In [14]:
def ComputingFrequenciesWithMismatches(Text, k, d):
    FrequencyArray = [0]* 4**k
    for i in range(0, len(Text)-k):
        Pattern = Text[i:k]
        neighborhood = Neighbors(Pattern, d)
        for ApproximatePattern in neighborhood:
            j = pattern_to_number(ApproximatePattern)
            FrequencyArray[j]+=1
    return FrequencyArray
            
def symbol_to_number(symbol):
    return {'A':0, 'C':1, 'G':2, 'T':3}[symbol]

def pattern_to_number(pattern):
    if not pattern:
        return 0
    symbol = pattern[-1]
    prefix = pattern[0:-1]
    return (4*pattern_to_number(prefix) + symbol_to_number(symbol))

<code>FrequentWordsWithMismatches(Text, k, d)
        FrequentPatterns ← an empty set
        Neighborhoods ← an empty list
        for i ← 0 to |Text| − k
            add Neighbors(Text(i, k), d) to Neighborhoods
        form an array NeighborhoodArray holding all strings in Neighborhoods
        for i ← 0 to |Neighborhoods| − 1
            Pattern ← NeighborhoodArray(i) 
            Index(i) ← PatternToNumber(Pattern)
            Count(i) ← 1
        SortedIndex ← Sort(Index)
        for i ← 0 to |Neighborhoods| − 2 
            if SortedIndex(i) = SortedIndex(i + 1)
                Count(i + 1) ← Count(i) + 1
       maxCount ← maximum value in array Count
       for i ← 0 to |Neighborhoods| − 1
           if Count(i) = maxCount
               Pattern ← NumberToPattern(SortedIndex(i), k)
               add Pattern to FrequentPatterns
       return FrequentPatterns </code>

In [15]:
def FrequentWordsWithMismatches(Text, k, d):
    counts = {}
    for i in range(len(Text)-k+1):
        for neighbor in neighbors(Text[i:i+k],d):
            if neighbor not in counts:
                counts[neighbor] = 0
            counts[neighbor] += 1
    m = max(counts.values())
    return [kmer for kmer in counts if counts[kmer] == m]

def neighbors( s, d ):
    if d == 0:
        return [s]
    if len(s) == 1:
        return ['A','C','G','T']
    out = []
    for neighbor in neighbors(s[1:],d):
        if hamming(s[1:],neighbor) < d:
            out.extend(['A'+neighbor,'C'+neighbor,'G'+neighbor,'T'+neighbor])
        else:
            out.append(s[0] + neighbor)
    return out

def hamming( s, t ):
    return sum([s[i] != t[i] for i in range(len(s))])

In [29]:
FrequentWordsWithMismatches('ACGTTGCATGTCGCATGATGCATGAGAGCT', 4, 1)

['ATGT', 'GATG', 'ATGC']

In [16]:
def frequentWordsWithMismatchesAndReverseComplements( s, k, d ):
    counts = {}
    for i in range(len(s)-k+1):
        for sub in [s[i:i+k],reverseComplement(s[i:i+k])]:
            for neighbor in neighbors(sub,d):
                if neighbor not in counts:
                    counts[neighbor] = 0
                counts[neighbor] += 1
    m = max(counts.values())
    return [kmer for kmer in counts if counts[kmer] == m]

def neighbors( s, d ):
    if d == 0:
        return [s]
    if len(s) == 1:
        return ['A','C','G','T']
    out = []
    for neighbor in neighbors(s[1:],d):
        if hamming(s[1:],neighbor) < d:
            out.extend(['A'+neighbor,'C'+neighbor,'G'+neighbor,'T'+neighbor])
        else:
            out.append(s[0] + neighbor)
    return out

def hamming( s, t ):
    return sum([s[i] != t[i] for i in range(len(s))])

def reverseComplement( s ):
    return ''.join([complement(s[i]) for i in range(len(s)-1,-1,-1)])

def complement( s ):
    return {'A':'T','T':'A','C':'G','G':'C'}[s]


In [34]:
frequentWordsWithMismatchesAndReverseComplements('ACGTTGCATGTCGCATGATGCATGAGAGCT', 4, 1)

['ACAT', 'ATGT']

In [40]:
len(frequentWordsWithMismatchesAndReverseComplements('aatgatgatgacgtcaaaaggatccgga\
taaaacatggtgattgcctcgcataacgcggtatgaaaatggattgaagcccgggccg\
tggattctactcaactttgtcggcttgagaaagacctgggatcctgggtat\
taaaaagaagatctatttatttagagatctgttctattgtgatctcttatta\
ggatcgcactgcccTGTGGATAAcaaggatccggcttttaagatcaacaacctggaaagga\
tcattaactgtgaatgatcggtgatcctggaccgtataagctgggatcag\
aatgaggggTTATACACAactcaaaaactgaacaacagttgttcTTTGGA\
TAActaccggttgatccaagcttcctgacagagTTATCCACAgtagatcg\
cacgatctgtatacttatttgagtaaattaacccacgatcccagccattc\
ttctgccggatcttccggaatgtcgtgatcaagaatgttgatcttcagtg'.upper(), 9, 1))

34

In [4]:
f = open('Salmonella_enterica.txt', 'r')
genome = f.read()

In [17]:
frequentWordsWithMismatchesAndReverseComplements(genome, 9, 1)

KeyError: '\n'

4877738